In [1]:
!pip install datasets
!apt install git-lfs

     |████████████████████████████████| 311 kB 5.1 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 243 kB 43.6 MB/s 
     |████████████████████████████████| 133 kB 44.5 MB/s 
     |████████████████████████████████| 1.1 MB 14.0 MB/s 
     |████████████████████████████████| 144 kB 55.9 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 271 kB 74.2 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,413 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and dir

In [2]:
import requests
from tqdm.notebook import tqdm

url_train = "https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
url_val = "https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"

json_train = requests.get(url_train).json()
json_val = requests.get(url_val).json()

In [ ]:
samples_train = json_train['data']
samples_val = json_val['data']

In [ ]:
'source', 'id', 'filename', 'story', 'questions', 'answers', 'name'

('source', 'id', 'filename', 'story', 'questions', 'answers', 'name')

In [ ]:
for x in samples_train:
    if any(['bad_turn' in y for y in x['answers']]):
        print(1)
        break

1


In [ ]:
TRAIN_DICT = {'source':[], 'id':[], 'story':[], 'questions':[], 'answers':[]}

for s in tqdm(samples_train):
    TRAIN_DICT['source'].append(s['source'])
    TRAIN_DICT['id'].append(s['id'])
    TRAIN_DICT['story'].append(s['story'])
    
    x = []
    for a in s['questions']:
        if 'bad_turn' in a:
            del a['bad_turn']
        x.append(a)
    TRAIN_DICT['questions'].append(x)

    y = []
    for a in s['answers']:
        if 'bad_turn' not in a:
            a['bad_turn'] = None
        y.append(a)
    TRAIN_DICT['answers'].append(y)


  0%|          | 0/7199 [00:00<?, ?it/s]

In [ ]:
VAL_DICT = {'source':[], 'id':[], 'story':[], 'questions':[], 'answers':[]}

for s in tqdm(samples_val):
    VAL_DICT['source'].append(s['source'])
    VAL_DICT['id'].append(s['id'])
    VAL_DICT['story'].append(s['story'])
    
    x = []
    for a in s['questions']:
        if 'bad_turn' in a:
            del a['bad_turn']
        x.append(a)
    VAL_DICT['questions'].append(x)

    y = []
    for a in s['answers']:
        if 'bad_turn' not in a:
            a['bad_turn'] = None
        y.append(a)
    VAL_DICT['answers'].append(y)


  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
from datasets import Dataset
import datasets

ds_t = Dataset.from_dict(T, split="train")
ds_v = Dataset.from_dict(V, split="validation")


In [ ]:
ds_t.features

{'answers': [{'bad_turn': Value(dtype='bool', id=None),
   'input_text': Value(dtype='string', id=None),
   'span_end': Value(dtype='int64', id=None),
   'span_start': Value(dtype='int64', id=None),
   'span_text': Value(dtype='string', id=None),
   'turn_id': Value(dtype='int64', id=None)}],
 'id': Value(dtype='string', id=None),
 'questions': [{'input_text': Value(dtype='string', id=None),
   'turn_id': Value(dtype='int64', id=None)}],
 'source': Value(dtype='string', id=None),
 'story': Value(dtype='string', id=None)}

In [ ]:
for x in ds_t:
    if any(['bad_turn' in y for y in x['answers']]):
        print(1)

In [ ]:
ds = datasets.DatasetDict({
    'train': ds_t,
    'validation': ds_v
})

ds.push_to_hub('coqa')

Pushing split train to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
c = datasets.load_dataset("alistvt/coqa")
c

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Using custom data configuration alistvt--coqa-ebb10d3ed3787ebf


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
ds_t[0]

{'answers': [{'bad_turn': None,
   'input_text': 'It was formally established in 1475',
   'span_end': 179,
   'span_start': 151,
   'span_text': 'Formally established in 1475',
   'turn_id': 1},
  {'bad_turn': None,
   'input_text': 'research',
   'span_end': 494,
   'span_start': 454,
   'span_text': 'he Vatican Library is a research library',
   'turn_id': 2},
  {'bad_turn': None,
   'input_text': 'history, and law',
   'span_end': 511,
   'span_start': 457,
   'span_text': 'Vatican Library is a research library for history, law',
   'turn_id': 3},
  {'bad_turn': None,
   'input_text': 'philosophy, science and theology',
   'span_end': 545,
   'span_start': 457,
   'span_text': 'Vatican Library is a research library for history, law, philosophy, science and theology',
   'turn_id': 4},
  {'bad_turn': None,
   'input_text': 'a  project',
   'span_end': 879,
   'span_start': 769,
   'span_text': 'March 2014, the Vatican Library began an initial four-year project of digitising its coll

In [ ]:
len(samples_train), len(samples_val)

(7199, 500)

In [ ]:
from pprint import pprint

pprint(samples_train[1])

{'answers': [{'input_text': 'Hard Rock Cafe',
              'span_end': 284,
              'span_start': 243,
              'span_text': "Hard Rock Cafe in New York's Times Square",
              'turn_id': 1},
             {'input_text': '$2 million.',
              'span_end': 210,
              'span_start': 180,
              'span_text': ' reaping a total $2 million. \n',
              'turn_id': 2},
             {'input_text': '$120,000',
              'span_end': 342,
              'span_start': 292,
              'span_text': ' pre-sale expectations of only $120,000 in sales. ',
              'turn_id': 3},
             {'input_text': 'Hoffman Ma',
              'span_end': 1365,
              'span_start': 1295,
              'span_text': 'Hoffman Ma, who bought the glove on behalf of '
                           'Ponte 16 Resort in Macau',
              'turn_id': 4},
             {'bad_turn': 'true',
              'input_text': 'Macau',
              'span_end': 1366,
      

In [ ]:
from tqdm.notebook import tqdm
columns = ["index", "name", "filename", "id", "source", "turn_id", "story", "question", "input_text", "span_text", "span_start", "span_end", "bad_turn"]

samples_train_list = []

for i, item in tqdm(enumerate(samples_train)):
    for answer, question in zip(item['answers'], item['questions']):
        d = {
            "index": i,
             "name": item["name"],
             "filename": item["filename"],
             "id": item["id"],
             "source": item["source"],
             "story": item["story"],
             "turn_id": answer["turn_id"],
             "question": question["input_text"],
             "input_text": answer["input_text"],
             "span_text": answer["span_text"],
             "span_start": answer["span_start"],
             "span_end": answer["span_end"],
             "bad_turn": answer.get("bad_turn", None) is not None
        }
        samples_train_list.append(d)

0it [00:00, ?it/s]

In [ ]:
len(samples_train_list)

108647

In [ ]:
from tqdm.notebook import tqdm
columns = ["index", "name", "filename", "id", "source", "turn_id", "story", "question", "input_text", "span_text", "span_start", "span_end", "bad_turn"]

samples_val_list = []

for i, item in tqdm(enumerate(samples_val)):
    for answer, question in zip(item['answers'], item['questions']):
        d = {
            "index": i,
             "name": item["name"],
             "filename": item["filename"],
             "id": item["id"],
             "source": item["source"],
             "story": item["story"],
             "turn_id": answer["turn_id"],
             "question": question["input_text"],
             "input_text": answer["input_text"],
             "span_text": answer["span_text"],
             "span_start": answer["span_start"],
             "span_end": answer["span_end"],
             "bad_turn": answer.get("bad_turn", None) is not None
        }
        samples_val_list.append(d)

0it [00:00, ?it/s]

In [ ]:
len(samples_val_list)

7983

In [ ]:
import pandas as pd

train_df = pd.DataFrame(data=samples_train_list)
val_df = pd.DataFrame(data=samples_val_list)

In [ ]:
train_df

,index,name,filename,id,source,story,turn_id,question,input_text,span_text,span_start,span_end,bad_turn
0,0,Vatican_Library.txt,Vatican_Library.txt,3zotghdk5ibi9cex97fepx7jetpso7,wikipedia,"The Vatican Apostolic Library (), more commonl...",1,When was the Vat formally opened?,It was formally established in 1475,Formally established in 1475,151,179,False
1,0,Vatican_Library.txt,Vatican_Library.txt,3zotghdk5ibi9cex97fepx7jetpso7,wikipedia,"The Vatican Apostolic Library (), more commonl...",2,what is the library for?,research,he Vatican Library is a research library,454,494,False
2,0,Vatican_Library.txt,Vatican_Library.txt,3zotghdk5ibi9cex97fepx7jetpso7,wikipedia,"The Vatican Apostolic Library (), more commonl...",3,for what subjects?,"history, and law",Vatican Library is a research library for hist...,457,511,False
3,0,Vatican_Library.txt,Vatican_Library.txt,3zotghdk5ibi9cex97fepx7jetpso7,wikipedia,"The Vatican Apostolic Library (), more commonl...",4,and?,"philosophy, science and theology",Vatican Library is a research library for hist...,457,545,False
4,0,Vatican_Library.txt,Vatican_Library.txt,3zotghdk5ibi9cex97fepx7jetpso7,wikipedia,"The Vatican Apostolic Library (), more commonl...",5,what was started in 2014?,a project,"March 2014, the Vatican Library began an initi...",769,879,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108642,7198,cnn_512d145c235e998513628c6c8501cdb63975d7f3.s...,cnn_512d145c235e998513628c6c8501cdb63975d7f3.s...,31qnsg6a5rtt5m7pens7xklnbwf87b,cnn,(CNN) -- Cristiano Ronaldo provided the perfec...,10,Who was a sub?,Xabi Alonso,substitute Xabi Alonso,1405,1427,False
108643,7198,cnn_512d145c235e998513628c6c8501cdb63975d7f3.s...,cnn_512d145c235e998513628c6c8501cdb63975d7f3.s...,31qnsg6a5rtt5m7pens7xklnbwf87b,cnn,(CNN) -- Cristiano Ronaldo provided the perfec...,11,Was it his first game this year?,Yes,Xabi Alonso made his first appearance of the ...,1415,1467,False
108644,7198,cnn_512d145c235e998513628c6c8501cdb63975d7f3.s...,cnn_512d145c235e998513628c6c8501cdb63975d7f3.s...,31qnsg6a5rtt5m7pens7xklnbwf87b,cnn,(CNN) -- Cristiano Ronaldo provided the perfec...,12,What position did the team reach?,third,Real moved up to third in the table,1520,1555,False
108645,7198,cnn_512d145c235e998513628c6c8501cdb63975d7f3.s...,cnn_512d145c235e998513628c6c8501cdb63975d7f3.s...,31qnsg6a5rtt5m7pens7xklnbwf87b,cnn,(CNN) -- Cristiano Ronaldo provided the perfec...,13,Who was ahead of them?,Barca.,six points behind Barca.,1557,1582,False


In [ ]:
sum(train_df["bad_turn"] == 1)

2093

In [ ]:
train_df.to_pickle("coqa_flat_train_df.pkl")
val_df.to_pickle("coqa_flat_val_df.pkl")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from datasets import Dataset
import datasets

ds_t = Dataset.from_pandas(train_df, split="train")
ds_v = Dataset.from_pandas(val_df, split="validation")

ds = datasets.DatasetDict({
    'train': ds_t,
    'validation': ds_v
})

ds.push_to_hub('coqa-flat')

Pushing split train to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split validation to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
d = datasets.load_dataset('alistvt/coqa-flat')

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Using custom data configuration alistvt--coqa-flat-859610cee67df2fb


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/16.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/alistvt--coqa-flat-859610cee67df2fb/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
d

DatasetDict({
    train: Dataset({
        features: ['index', 'name', 'filename', 'id', 'source', 'story', 'turn_id', 'question', 'input_text', 'span_text', 'span_start', 'span_end', 'bad_turn'],
        num_rows: 108647
    })
    validation: Dataset({
        features: ['index', 'name', 'filename', 'id', 'source', 'story', 'turn_id', 'question', 'input_text', 'span_text', 'span_start', 'span_end', 'bad_turn'],
        num_rows: 7983
    })
})